# Seq2Seq Translator

In the previous notebook, I underwent the task of preprocessing the data. Namely for Italian-English translation.

I ought to do the same for French.. though I am not currently keen on spending another $82$ mins to lemmatise all $900k$ lines of French-English.

In this notebook, I will be building a Seq2Seq Architecture for Machine Translation; said architecture will undergo the same cycle as all models usually do. From training, evaluating, validating to testing.